In [ ]:
# This section installs packages needed to run the script. 

import folium  # Importing GeoJson class from folium library for visualizing geospatial data
import pandas as pd  # Importing pandas library and aliasing it as pd for data analysis and manipulation
import matplotlib.pyplot as plt  # Importing pyplot module from matplotlib library and aliasing it as plt for plotting
import scipy.signal  # Importing signal module from scipy library for signal processing
import numpy as np  # Importing numpy library and aliasing it as np for numerical computing
import geopandas as gpd  # Importing geopandas library and aliasing it as gpd for working with geographic data
import openeo  # Importing openeo library for working with the OpenEO API for Earth Observation data processing
from shapely.geometry import shape  # Importing shape class from geometry module of shapely library for geometric operations


In [ ]:
#This establishes a connection with the Sentenel satelite data. 

connection = openeo.connect(url="openeo.dataspace.copernicus.eu")
connection.authenticate_oidc()

#if this does not read as 'Authorised successfully' or 'Authenticated using refresh token', then please ensure that you have completed the setup steps as outlined in section 2.6 of the user guide. 
#if you have followed the steps in section 2.6 correctly and the problem persists, please look at https://dataspace.copernicus.eu/news for any information about service interruptions. 
#if there is no news of service problems you can raise a ticket here: https://helpcenter.dataspace.copernicus.eu/hc/en-gb/requests/new

In [ ]:
# This section displays a list of all the PreZero landfill sites

landfills = gpd.read_file(r"C:\GIS_Course\EGM722\Project\Data\PZ_landfill_point4326.geojson")

landfills

In [ ]:
# This section displays a map of all the landfill sites. Click any site for its name. 

# This creates the map and centres it on the geometries. 
centroids = landfills.geometry.centroid
center = [centroids.y.mean(), centroids.x.mean()]
site_map = folium.Map(location=center, zoom_start=5)

# Adding the landfill locations to the map
for feature in landfills.iterfeatures():
    # Extract feature number from properties
    #feature_number = feature['properties']['DIRECCION'] #this is for the test landfill
    feature_number = feature['properties']['Landfill'] #this is for the PreZero ones
    
    # Extract coordinates of the feature
    coordinates = feature['geometry']['coordinates']
    
    # Create a marker with label for each feature
    folium.Marker(location=[coordinates[1], coordinates[0]], 
                  popup=f"Feature {feature_number}").add_to(site_map)

# Display the map
site_map

# the warning "...Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect..." can be ignored. 

In [ ]:
# This section applies a buffer of 5000m to each landfill for the analysis.

# loading dataframe
landfill_5000m = gpd.read_file(r"C:\GIS_Course\EGM722\Project\Data\PZ_landfill_point4326.geojson")

# The dataset is projected in EPSG:4326 with its units in degrees. This needs to be converted to CRS to EPSG:2062, which is in metres. 
landfill_5000m = landfill_5000m.to_crs(epsg=2062)

# Now the dataframe is converted, a buffer of 5000m is added to each point
landfill_5000m['geometry'] = landfill_5000m.buffer(5000)

# Now the buffered data needs to be converted back to EPSG:4326 because the Sentinel data is projected in EPSG:4326.
landfill_5000m = landfill_5000m.to_crs(epsg=4326)

# The time series analysis requires that A GeoJSON format file is used for the analised areas, so this bit produces a file suitable for that.
landfill_5000m_geojson = landfill_5000m.__geo_interface__



In [ ]:
# This selects the specific dataset for the time series analysis. 

s5cube_timeseries = connection.load_collection(
    "SENTINEL_5P_L2",  
    temporal_extent=["2021-08-01", "2021-10-31"], # Select a date range using the format YYYY-MM-DD
    bands=["CH4"], # The options for gas monitoring are: 'CO', 'HCHO', 'NO2', 'O3', 'SO2', 'CH4'
)

In [ ]:
# This section takes the mean values of the selected gas within each of the buffered landfill geometries for every dataset within the selected time period. 
# Sentinel 5P will provide daily values of any gas you select. 

timeseries = s5cube_timeseries.aggregate_spatial(geometries=landfill_5000m_geojson, reducer="mean")

#This saves the results as a .CSV file which can be viewed in Microsoft Excel or a similar package. It will be saved in the indicated location.
job = timeseries.execute_batch(out_format="CSV", title="Gas timeseries")

job.get_results().download_file("Gas_Timeseries_results/Gas_timeseries.csv")

pd.read_csv("Gas_Timeseries_results/Gas_timeseries.csv", index_col=0)

In [ ]:
#This section defines the graph where the time series is plotted

def plot_timeseries(filename, figsize=(15, 10)): #here you can specify how big the graph is on the screen
    df = pd.read_csv(filename, index_col=0)
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()

    fig, ax = plt.subplots(figsize=figsize, dpi=90)
    df.groupby("feature_index")["avg(band_0)"].plot(marker="o", ax=ax)
    ax.set_title(filename.split("/")[-1])
    ax.set_ylabel("Parts per billion for CH4 or mol/m2 for all other gasses")
    
    # Calculate the minimum and maximum values
    ymin = df["avg(band_0)"].min()
    ymax = df["avg(band_0)"].max()

    # Add 10% to the minimum and maximum values
    ymin_with_margin = ymin - 0.1 * (ymax - ymin)
    ymax_with_margin = ymax + 0.1 * (ymax - ymin)

    ax.set_ylim(ymin_with_margin, ymax_with_margin)
    
    ax.legend(title="parcel id", loc='upper left', bbox_to_anchor=(1.02, 1), ncol=2)
    ax.xaxis.set_major_locator(plt.MaxNLocator(30))
    ax.grid(True)

plot_timeseries("Gas_Timeseries_results/Gas_timeseries.csv")

#CH4 measurements are column averaged dry air mixing ratio of methane (parts per billion). Ranges of 1,600 - 2,000
#NO2 measurements are Nitrogen dioxide tropospheric column (mol/m2). Ranges of 0 - 0.0003
#CO measurements are Carbon monoxide total column (mol/m2). Ranges of 0 - 0.1
#HCHO measurements are Formaldehyde troposheric vertical column (mol/m2). Ranges of 0 - 0.001
#O3 measurements are Ozone total column (mol/m2). Ranges of 0 - 0.36
#SO2 measurements are Sulfur dioxide total column (mol/m2). Ranges of 0 - 0.01
#More information: https://docs.sentinel-hub.com/api/latest/data/sentinel-5p-l2/